In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import requests

from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from io import BytesIO
from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =
        
        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(src,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                pic = Image(BytesIO(resp.content))

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

总数量：211
[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1185_TOP_P04.jpg
[剩余数量：181] - [当前时间：21:59:21]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1204_TOP_P04.jpg
[剩余数量：180] - [当前时间：21:59:24]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1179_TOP_P04.jpg
[剩余数量：179] - [当前时间：21:59:25]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1206_TOP_P04.jpg
[剩余数量：178] - [当前时间：21:59:25]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1180_TOP_P04.jpg
[剩余数量：177] - [当前时间：21:59:26]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1173_TOP_P04.jpg
[剩余数量：176] - [当前时间：21:59:27]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1194_TOP_P04.jpg
[剩余数量：175] - [当前时间：21:59:28]

[状态：ok，尝试次数：1] - https://ecat.spectrapremium.com/img/produits/TB1147_TOP_P04.jpg
[剩余数量：174] - [当前时间：21:59:29]

[状态：ok，尝试次数：4] - https://ecat.spectrapremium.com/img/produits/TB1151_TOP_P04.jpg
[剩余数量：173] - [当前时间：21:5